<a href="https://colab.research.google.com/github/UWIZEYEDelphine/enhanced-densenet/blob/main/foodmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%rm -rf <uef_60>

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `rm -rf <uef_60>'


In [ ]:
import numpy as np
import pathlib
import tensorflow as tf

In [ ]:
densenet=tf.keras.applications.densenet.DenseNet201(weights='imagenet',include_top=False,input_shape=(300,300,3))
for layer in densenet.layers[:120]:
  layer.trainable=False
model=tf.keras.Sequential([
                  densenet,
                  tf.keras.layers.Flatten(),
                  tf.keras.layers.Dropout(0.5),
                  tf.keras.layers.Dense(512, activation='relu'),
                  tf.keras.layers.Dense(256,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01),bias_regularizer=tf.keras.regularizers.l2(0.01)),
                  tf.keras.layers.Dense(256, activation='softmax'),
                  
])


In [ ]:
dir1='/content/delphine/UECF_256_CROP'

gen1=tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1./255,width_shift_range=0.3,zoom_range=0.3,height_shift_range=0.2,rotation_range=30,validation_split=0.4)
gen2=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.4)

training_data=gen1.flow_from_directory(dir1, batch_size=16, target_size=(300,300),shuffle=True,subset='training')
validation_data=gen2.flow_from_directory(dir1, batch_size=16, target_size=(300,300), subset='validation')
print(training_data.image_shape)

Found 18939 images belonging to 256 classes.
Found 12456 images belonging to 256 classes.
(300, 300, 3)


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
met=tf.keras.metrics.TopKCategoricalAccuracy(k=1)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=met)
history=model.fit(training_data, epochs=40, verbose=1, validation_data=validation_data,steps_per_epoch=1190, validation_steps=800)

Epoch 1/40
1190/1190 [==============================] - 894s 751ms/step - loss: 6.1616 - top_k_categorical_accuracy: 0.1388 - val_loss: 4.3072 - val_top_k_categorical_accuracy: 0.2677
Epoch 2/40
1190/1190 [==============================] - 622s 522ms/step - loss: 3.3445 - top_k_categorical_accuracy: 0.3535 - val_loss: 3.2825 - val_top_k_categorical_accuracy: 0.4155
Epoch 3/40
1190/1190 [==============================] - 619s 520ms/step - loss: 2.5111 - top_k_categorical_accuracy: 0.4667 - val_loss: 2.8779 - val_top_k_categorical_accuracy: 0.4804
Epoch 4/40
1190/1190 [==============================] - 620s 521ms/step - loss: 2.0736 - top_k_categorical_accuracy: 0.5453 - val_loss: 2.3073 - val_top_k_categorical_accuracy: 0.5442
Epoch 5/40
1190/1190 [==============================] - 621s 522ms/step - loss: 1.7873 - top_k_categorical_accuracy: 0.5967 - val_loss: 2.3712 - val_top_k_categorical_accuracy: 0.5630
Epoch 6/40
1190/1190 [==============================] - 621s 522ms/step - loss: 

In [ ]:
saved_model_dir = '/content/delphine/models/Densenet201-model2'
tf.saved_model.save(model, saved_model_dir)

In [ ]:
model= tf.keras.models.load_model('/content/delphine/models/Densenet201-model2')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/delphine/models/Densenet201-model2')
tflite_model = converter.convert()

with open('/content/delphine/foodmodel.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
import os

labels = '\n'.join(sorted(training_data.class_indices.keys()))

with open('/content/delphine/labels.txt', 'w') as f:
  f.write(labels)